str/byte로 변환한 dom을 실제 선택가능한 dom object로 바꿔주기

(1) 정규식  
(2) beautifulsoup

여기선 beautifulsoup만 해보겠음

In [89]:
from bs4 import BeautifulSoup

html = """
<html>
    <head></head>
    <body>
        <div id="thisIsDiv">This is Div
            <p>
                <a class="red" id = "a1" detail = "veryRed">This is veryRed text</a>
                <a class="red" id = "a2" detail = "darkishRed">
                    <a class="red" id = "a3" detail = "paleRed"></a>
                </a>
            </p>
        </div>
    </body>
</html>
"""

dom = BeautifulSoup(html, "lxml")

참고로 beautifulsoup 에서 여러가지 parser를 쓸 수 있고, 이에 따라 성능이 달라지곤 하는데<br />• lxml        : 가장 빠르지만, welformed에서만 잘 작동을 함<br />• html.parser : welformed 가 아니라도 그냥저냥 작동함  `from html.parser import HTMLParser`

dom객체를 select하는 다양한 방법이 있음<br />

① 자식 - tag name, attribute, text로 찾기<br />
`find(name, attrs, recursive, string, **kwargs)
 find_all(name, attrs, recursive, string, **kwargs)`<br />

② 부모/형제 - 관계로 찾기<br />
`find_parent
 find_parents
 find_next_sibling
 find_previous_sibling
 find_next_siblings
 find_previous_siblings`<br />

③`dom.select`로 찾기 - select내부에서의 문법<br />
• ID    : #<br />
• class : .<br />
• 자손   : 띄어쓰기<br />
• 자식   : ><br />
※ 참고 : select 외부에서는 . 을 찍으면 자손을 의미함

In [68]:
# ① find - tag name
dom.div.p.find('a')

<a class="red" detail="veryRed" id="a1">This is veryRed text</a>

In [55]:
# ① find - attribute
dom.find('a', {"class" : "red", "id" : "a2"}).find_previous_sibling()

<a class="red" detail="veryRed" id="a1">This is veryRed text</a>

In [57]:
# ① find - text
dom.find('a',text='This is veryRed text')

<a class="red" detail="veryRed" id="a1">This is veryRed text</a>

In [69]:
dom.select("a#a2")

[<a class="red" detail="darkishRed" id="a2">
 </a>]

In [71]:
dom.select("div#thisIsDiv")

[<div id="thisIsDiv">
 <p>
 <a class="red" detail="veryRed" id="a1">This is veryRed text</a>
 <a class="red" detail="darkishRed" id="a2">
 </a><a class="red" detail="paleRed" id="a3"></a>
 </p>
 </div>]

In [44]:
dom.select("body p")

[<p>
 <a class="red" detail="veryRed" id="1">This is veryRed text</a>
 <a class="red" detail="darkishRed" id="2">
 </a><a class="red" detail="paleRed" id="3"></a>
 </p>]

In [43]:
dom.select("div>p")

[<p>
 <a class="red" detail="veryRed" id="1">This is veryRed text</a>
 <a class="red" detail="darkishRed" id="2">
 </a><a class="red" detail="paleRed" id="3"></a>
 </p>]

In [74]:
# dom.select(".rc > .r a") 요기선 안되는데 이런 복잡한 것도...

In [87]:
import requests
import time
from bs4 import BeautifulSoup

headers = {'user-agent':"Mozilla/5.0 (Macintosh; Intel Mac OS X 10_14_5) AppleWebKit/537.36 (KHTML, like Gecko) Chrome/73.0.3683.103 Safari/537.36"}
def download(method, url, params=None, data=None, timeout=60, maxretry=3):
    try:            
        resp = requests.request(method, url, params=params, data=data, headers=headers) 
        resp.raise_for_status()
    except requests.exceptions.HTTPError as e:
        if 500 <= e.response.status_code < 600 and maxtries > 0:
            time.sleep(timeout)
            download(method, url, params, data, timeout, maxretry-1)
        else:      
            print(e.response.status_code)
            print(e.response.reason)
    return resp

# 네이버 - 쉐이크쉑 검색결과 가져옴
url = "https://search.naver.com/search.naver?sm=tab_hty.top&where=nexearch&query=%EC%89%90%EC%9D%B4%ED%81%AC%EC%89%91&oquery=%EB%B0%95%EB%B3%B4%EC%98%81&tqi=UflIysprvN8ssZLPg30ssssstsd-205107"
html = download('get', url)
dom = BeautifulSoup(html.text, "lxml")
count = 0
for row in dom.select('a.title_link'):
    count += 1
    print(f"----------{count}--------------")
    print(row.text)
    
print(" ")
print(" ")

# 구글 - 박보영
url = "https://www.google.com/search?q=%EB%B0%95%EB%B3%B4%EC%98%81&rlz=1C5CHFA_enKR841KR841&oq=%EB%B0%95%EB%B3%B4%EC%98%81&aqs=chrome..69i57j69i59j0l4.1851j0j7&sourceid=chrome&ie=UTF-8"
html = download('get', url)
dom = BeautifulSoup(html.text, "lxml")
for row in dom.select('h3.LC20lb'):
    print(row.text)
    
print(" ")
print(" ")    

# 다음 - 초콜릿
url = "https://search.daum.net/search?w=tot&DA=YZR&t__nil_searchbox=btn&sug=&sugo=&q=%EC%B4%88%EC%BD%9C%EB%A6%BF"
html = download('get', url)
dom = BeautifulSoup(html.text, "lxml")
count = 0
for row in dom.select('a.f_link_b'):
    count += 1
    print(f"----------{count}--------------")
    print(row.text)

print(" ")
print(" ")

# 네이버 - 이미지검색의 링크들
url = "https://search.naver.com/search.naver?sm=tab_hty.top&where=image&query=%EB%B0%95%EB%B3%B4%EC%98%81&oquery=%EB%B0%95%EB%B3%B4%EC%98%81%E3%85%87&tqi=Ufloswp0J1sss4H9k5KssssstWl-350369"
html = download('get', url)
dom = BeautifulSoup(html.text, "lxml")
count = 0
for row in dom.find_all("a", {"class":"thumb _thumb"}):
    print(f"--------count = {count}------------")
    print(row.find_all("img")[0]["data-source"])
    count+=1
    
# 물론 정규식으로도 가능함
import re
re.findall('<img src=".+?" class="_img" alt = "(.+?)" onerror=".+?" data-source="(.+?)" data-width="\d+?" data-height="/d+">',html.text)


----------1--------------
쉐이크쉑
----------2--------------
쉐이크 쉑 - 나무위키
----------3--------------
Shake Shack Korea - 쉐이크쉑 - 홈 | Facebook
 
 
박보영 - 위키백과, 우리 모두의 백과사전
박보영의 작품 목록 - 위키백과, 우리 모두의 백과사전
박보영은 오래 지켜본다. 연애도, 연기 변신도 - 중앙일보
#박보영 hashtag on Twitter
박보영 - Daum 영화
전체글 - 박보영 공식 팬카페 _ 뽀르테 - Daum 카페
박보영, tvN 드라마 '어비스' 여주인공 - MSN.com
'너의 결혼식' → '어비스' 박보영, 근황 살펴보니…“여전히 뽀블리”
 
 
----------1--------------
'바람이분다' 김하늘, 감우성 초콜릿 특허 되찾았다 [결정적장면]
----------2--------------
하지원, “드라마 초콜릿 촬영중..
----------3--------------
털실무스케이크 얼그레이 맛 vs 다크초콜릿 맛, 용산의 명소(생활의...
----------4--------------
[EBS 금요극장] 영화 '초콜릿'
----------5--------------
[6/20일 수달이가 물어온] 초콜릿색을 띈!!!"초코오징어...
----------6--------------
이강인 초콜릿 논란 밝혀짐 ㅋㅋㅋㅋ
----------7--------------
초보도 쉽게 만들수 있는 '초콜릿 휘낭시에' (+영상레시피)
----------8--------------
사 온 발렌타인 데이 초콜릿 아빠가 실수로 먹었는데...
----------9--------------
트로피코6 초콜릿 공장
----------10--------------
리터스포트 미니 초콜릿♪
----------11--------------
진한 초콜릿 파운드케이크
----------12--------------
수제초콜릿, 김보람초콜릿
 
 
--------